All data downloaded from https://www.football-data.co.uk/spainm.php

In [2]:
import numpy as np
import pandas as pd

In [18]:
league_names = ['primera', 'segunda']
league_years = [f"{year}_{year+1}" for year in range(1999, 2023)]

In [21]:
df_list = dict()
for league_year in league_years:
    for league_name in league_names:
        cur_league = f"{league_name}_{league_year}"
        df_list[cur_league] = pd.read_csv("./initial_ds/" + cur_league + ".csv")
        
print(f"{len(df_list)} dataframes were read.")

df_list.keys()

48 dataframes were read.


dict_keys(['primera_1999_2000', 'segunda_1999_2000', 'primera_2000_2001', 'segunda_2000_2001', 'primera_2001_2002', 'segunda_2001_2002', 'primera_2002_2003', 'segunda_2002_2003', 'primera_2003_2004', 'segunda_2003_2004', 'primera_2004_2005', 'segunda_2004_2005', 'primera_2005_2006', 'segunda_2005_2006', 'primera_2006_2007', 'segunda_2006_2007', 'primera_2007_2008', 'segunda_2007_2008', 'primera_2008_2009', 'segunda_2008_2009', 'primera_2009_2010', 'segunda_2009_2010', 'primera_2010_2011', 'segunda_2010_2011', 'primera_2011_2012', 'segunda_2011_2012', 'primera_2012_2013', 'segunda_2012_2013', 'primera_2013_2014', 'segunda_2013_2014', 'primera_2014_2015', 'segunda_2014_2015', 'primera_2015_2016', 'segunda_2015_2016', 'primera_2016_2017', 'segunda_2016_2017', 'primera_2017_2018', 'segunda_2017_2018', 'primera_2018_2019', 'segunda_2018_2019', 'primera_2019_2020', 'segunda_2019_2020', 'primera_2020_2021', 'segunda_2020_2021', 'primera_2021_2022', 'segunda_2021_2022', 'primera_2022_2023', 's